In [1]:
import sqlite3

In [2]:
romanian_texts = {}
moldavian_texts = {}

conn = sqlite3.connect('news.db')
c = conn.cursor()

c.execute('SELECT * FROM romania')
rows = c.fetchall()
for row in rows:
    if row[4] not in romanian_texts:
        romanian_texts[row[4]] = []
    romanian_texts[row[4]].append(row[5])
    
c.execute('SELECT * FROM moldova WHERE newspaper != "zugo"')
rows = c.fetchall()
for row in rows:
    text = ''
    if len(row[5]) > 10000:
        text = row[5][:10000]
    else:
        text = row[5]
    if row[4] not in moldavian_texts:
        moldavian_texts[row[4]] = []
        
    moldavian_texts[row[4]].append(text)

conn.close()

In [5]:
all_texts = {"romana": [], "moldova": []}

for key in romanian_texts:
    all_texts["romana"].extend(romanian_texts[key])

    
for key in moldavian_texts:
    all_texts["moldova"].extend(moldavian_texts[key])

In [7]:
!python -m spacy download ro_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 9.5 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_md')


In [29]:
import spacy

nlp = spacy.load('ro_core_news_md')

all_texts["romana"] = [nlp(text) for text in all_texts["romana"]]
all_texts["moldova"] = [nlp(text) for text in all_texts["moldova"]]


# Dependency tree
all_dependencies = {"romana": [], "moldova": []}

for text in all_texts["romana"]:
    for sent in text.sents:
        token_dep_list = []
        for token in sent:
            token_dep_list.append(token.dep_)
        all_dependencies["romana"].append(token_dep_list)

for text in all_texts["moldova"]:
    for sent in text.sents:
        token_dep_list = []
        for token in sent:
            token_dep_list.append(token.dep_)
        all_dependencies["moldova"].append(token_dep_list)
    

In [55]:
# Most common syntactic dependencies
from collections import Counter

most_common_dependencies = {"romana": [], "moldova": []}

deps = {}
for sent in all_dependencies["romana"]:
    for dep in sent:
        if dep not in deps:
            deps[dep] = 2**len(deps)
        
for sent in all_dependencies["moldova"]:
    for dep in sent:
        if dep not in deps:
            deps[dep] = 2**len(deps)
    
sentence_numbers = {}        
sentences = {}
for sent in all_dependencies["romana"]:
    sentence = [tag for tag in sent]
    sum = 0
    for tag in sentence:
        sum += deps[tag]
    if sum not in sentence_numbers:
        sentence_numbers[sum] = 0
    else:
        sentence_numbers[sum] += 1
        
    if sum not in sentences:
        sentences[sum] = sent
    
print("Romana")  
sorted_sent_numbers = sorted(sentence_numbers, reverse=True)
mc_romana = sorted_sent_numbers[:3]
for mc in mc_romana:
    spacy.displacy.render(nlp(' '.join(sentences[mc])), style='dep', jupyter=True)
    
sentence_numbers = {}        
sentences = {}
for sent in all_dependencies["moldova"]:
    sentence = [tag for tag in sent]
    sum = 0
    for tag in sentence:
        sum += deps[tag]
    if sum not in sentence_numbers:
        sentence_numbers[sum] = 0
    else:
        sentence_numbers[sum] += 1
        
    if sum not in sentences:
        sentences[sum] = sent
    
print("Moldova")
sorted_sent_numbers = sorted(sentence_numbers, reverse=True)
mc_moldova = sorted_sent_numbers[:3]
for mc in mc_moldova:
    spacy.displacy.render(nlp(' '.join(sentences[mc])), style='dep', jupyter=True)


Romana


Moldova
